In [1]:
import sys
sys.path.append('../framework')
from NetworkClass import Network
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision
import torch
import torch.nn as nn
import copy


In [2]:
class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size

    def __call__(self, img):
        return torch.reshape(img, self.new_size)

In [3]:
# Define Constants
batch_size_train = 64
batch_size_test = 1000
n_epochs = 3
learning_rate = 0.01
momentum = 0.5
log_interval = 10
random_seed = 1

In [4]:
# Model Definition
model_dict = {
        "network":{
            'input_layer': {
                "units": 784,
                
                },
            'hidden_layer': [{
                    "units": 400, 
                    "type": "Linear"
                }, 
                {
                    "units": 50, 
                    "activation": "relu",
                    "type": "Linear"

                }],
            'output_layer': {
                "units": 10,
                "activation": "softmax",
                "type": "Linear"
                }
        }
    }

model = Network(model_dict)

In [6]:
for (layer, param) in enumerate(model.parameters()):
    print("Layer {} , Parameters: {}".format(layer, param.shape))

Layer 0 , Parameters: torch.Size([400, 784])
Layer 1 , Parameters: torch.Size([400])
Layer 2 , Parameters: torch.Size([50, 400])
Layer 3 , Parameters: torch.Size([50])
Layer 4 , Parameters: torch.Size([10, 50])
Layer 5 , Parameters: torch.Size([10])


In [7]:
# Load Datasets

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../data/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,)), ReshapeTransform((-1,))
                             ])),
  batch_size=batch_size_train, shuffle=True)


test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../data/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,)), ReshapeTransform((-1,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [10]:
# Define Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [13]:
def train(mod, optim, epoch):
  mod.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optim.zero_grad()
    output = mod(data)
    loss = criterion(output, target)
    loss.backward()
    optim.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [14]:
def test(mod):
  mod.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = mod(data)
      test_loss += criterion(output, target).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [15]:
test(model)

# Pruning every 3 epochs
for epoch in range(1, n_epochs + 1):
  train(model, optimizer, epoch)
  test(model)
    


../framework/NetworkClass.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activation_scheme[-1](self.output_layer(x))



Test set: Avg. loss: 0.0023, Accuracy: 1209/10000 (12%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303816
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.301586
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.300818
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.300153
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.300347
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.300650
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.300638
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.301147
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.301393
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.300672
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.301821
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.299628
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.300310
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.298480
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.299376
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.298394
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.298184
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.297325
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.297630
Train Epoch: 1 [12160

Train Epoch: 2 [44160/60000 (74%)]	Loss: 1.814335
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.794509
Train Epoch: 2 [45440/60000 (76%)]	Loss: 1.849588
Train Epoch: 2 [46080/60000 (77%)]	Loss: 1.818519
Train Epoch: 2 [46720/60000 (78%)]	Loss: 1.822612
Train Epoch: 2 [47360/60000 (79%)]	Loss: 1.884128
Train Epoch: 2 [48000/60000 (80%)]	Loss: 1.838719
Train Epoch: 2 [48640/60000 (81%)]	Loss: 1.782363
Train Epoch: 2 [49280/60000 (82%)]	Loss: 1.729094
Train Epoch: 2 [49920/60000 (83%)]	Loss: 1.806979
Train Epoch: 2 [50560/60000 (84%)]	Loss: 1.809748
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.820247
Train Epoch: 2 [51840/60000 (86%)]	Loss: 1.815244
Train Epoch: 2 [52480/60000 (87%)]	Loss: 1.833672
Train Epoch: 2 [53120/60000 (88%)]	Loss: 1.767771
Train Epoch: 2 [53760/60000 (90%)]	Loss: 1.806596
Train Epoch: 2 [54400/60000 (91%)]	Loss: 1.761911
Train Epoch: 2 [55040/60000 (92%)]	Loss: 1.782087
Train Epoch: 2 [55680/60000 (93%)]	Loss: 1.727672
Train Epoch: 2 [56320/60000 (94%)]	Loss: 1.804329


In [16]:
def prune_neurons(nn_model):
    print("Pre-Pruning/n")
    for (layer, param) in enumerate(nn_model.parameters()):
        print("Layer {} , Parameters: {}".format(layer, param.shape))    
        
    # 1 neuron is pruned from each layer based on the minimum L1 norm.
    neurons_to_prune = []
    for p in nn_model.parameters():
            if len(p.data.size()) != 1:
                normed_weights = p.data.abs()
                l1_norm_layer = []
                for neuron_idx in range(normed_weights.shape[0]):
                    l1_norm_layer.append(torch.sum(normed_weights[neuron_idx, :]).item())
                neurons_to_prune.append(torch.argmin(torch.FloatTensor(l1_norm_layer)))
                neurons_to_prune.append(torch.argmin(torch.FloatTensor(l1_norm_layer)))

                
    # Modify the model parameters to update the shape of the network after pruning
    param_list = list(nn_model.parameters())
    for i, neuron_idx in enumerate(neurons_to_prune):
        idx_weights = param_list[i]
        if i < len(param_list) - 2:
            y = torch.cat((idx_weights[0:neuron_idx], idx_weights[neuron_idx+1:]))
            if i > 1 and len(idx_weights.shape) > 1:

                y = torch.cat((y[:, 0:neuron_idx-1], y[:, neuron_idx:]), axis=1)
        elif i > 1 and len(idx_weights.shape) > 1:
            y = torch.cat((idx_weights[:, 0:neuron_idx-1], idx_weights[:, neuron_idx:]), axis=1)
        else:
            y = idx_weights
        idx_weights.data = y  
       
    print("Post Pruning /n")
    for (layer, param) in enumerate(nn_model.parameters()):
        print("Layer {} , Parameters: {}".format(layer, param.shape))   
        
    return nn_model

In [17]:
def prune_train(model, optimizer):
    new_model = copy.deepcopy(model)
    new_optimizer = copy.deepcopy(optimizer)
    
    # Get new model after pruning
    new_model = prune_neurons(new_model)
    new_model_dict = copy.deepcopy(model_dict)
    
    
    for i, layer in enumerate(model_dict['network']['hidden_layer']):
        if i == 0:

            new_model_dict['network']['hidden_layer'][i]['units'] = new_model.state_dict()['input_layer.weight'].shape[0]
        else:
            new_model_dict['network']['hidden_layer'][i]['units'] = new_model.state_dict()['hidden_layers.{}.weight'.format(i-1)].shape[0]

    # Create new model based on updated network definition
    updated_model = Network(new_model_dict)
    
    # Load previously trained parameters into the new model
    updated_model.load_state_dict(new_model.state_dict())
    
    # Update Optimizers and set state from the previous optimizer
    criterion = nn.CrossEntropyLoss()
    updated_optimizer = torch.optim.SGD(updated_model.parameters(), lr=learning_rate)
    updated_optimizer.load_state_dict(new_optimizer.state_dict())
    
    # Train and test the model
    for epoch in range(1, n_epochs + 1):
      train(updated_model, updated_optimizer, epoch)
      test(updated_model)
        
    return updated_model, updated_optimizer

In [18]:
# Perform Pruning 5 times in succession
for i in range(5):
    model, optimizer = prune_train(model, optimizer)


Pre-Pruning/n
Layer 0 , Parameters: torch.Size([400, 784])
Layer 1 , Parameters: torch.Size([400])
Layer 2 , Parameters: torch.Size([50, 400])
Layer 3 , Parameters: torch.Size([50])
Layer 4 , Parameters: torch.Size([10, 50])
Layer 5 , Parameters: torch.Size([10])
Post Pruning /n
Layer 0 , Parameters: torch.Size([399, 784])
Layer 1 , Parameters: torch.Size([399])
Layer 2 , Parameters: torch.Size([49, 399])
Layer 3 , Parameters: torch.Size([49])
Layer 4 , Parameters: torch.Size([10, 49])
Layer 5 , Parameters: torch.Size([10])
softmax
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.173197
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.166616
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.120072
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.068549
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.044531
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.011092
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.985745
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.932876
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.900892
Train Epoch: 1 [5760/60000 (10%)]	

Train Epoch: 2 [39040/60000 (65%)]	Loss: 1.748442
Train Epoch: 2 [39680/60000 (66%)]	Loss: 1.750717
Train Epoch: 2 [40320/60000 (67%)]	Loss: 1.713986
Train Epoch: 2 [40960/60000 (68%)]	Loss: 1.707956
Train Epoch: 2 [41600/60000 (69%)]	Loss: 1.696465
Train Epoch: 2 [42240/60000 (70%)]	Loss: 1.677268
Train Epoch: 2 [42880/60000 (71%)]	Loss: 1.738071
Train Epoch: 2 [43520/60000 (72%)]	Loss: 1.752095
Train Epoch: 2 [44160/60000 (74%)]	Loss: 1.725630
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.718251
Train Epoch: 2 [45440/60000 (76%)]	Loss: 1.749370
Train Epoch: 2 [46080/60000 (77%)]	Loss: 1.695526
Train Epoch: 2 [46720/60000 (78%)]	Loss: 1.660160
Train Epoch: 2 [47360/60000 (79%)]	Loss: 1.755663
Train Epoch: 2 [48000/60000 (80%)]	Loss: 1.829916
Train Epoch: 2 [48640/60000 (81%)]	Loss: 1.621207
Train Epoch: 2 [49280/60000 (82%)]	Loss: 1.631159
Train Epoch: 2 [49920/60000 (83%)]	Loss: 1.710237
Train Epoch: 2 [50560/60000 (84%)]	Loss: 1.671974
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.806259


Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.765574
Train Epoch: 1 [16640/60000 (28%)]	Loss: 1.810512
Train Epoch: 1 [17280/60000 (29%)]	Loss: 1.738834
Train Epoch: 1 [17920/60000 (30%)]	Loss: 1.740834
Train Epoch: 1 [18560/60000 (31%)]	Loss: 1.801771
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.736371
Train Epoch: 1 [19840/60000 (33%)]	Loss: 1.782035
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.749394
Train Epoch: 1 [21120/60000 (35%)]	Loss: 1.774279
Train Epoch: 1 [21760/60000 (36%)]	Loss: 1.747349
Train Epoch: 1 [22400/60000 (37%)]	Loss: 1.680492
Train Epoch: 1 [23040/60000 (38%)]	Loss: 1.808181
Train Epoch: 1 [23680/60000 (39%)]	Loss: 1.770405
Train Epoch: 1 [24320/60000 (41%)]	Loss: 1.752162
Train Epoch: 1 [24960/60000 (42%)]	Loss: 1.707279
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.778734
Train Epoch: 1 [26240/60000 (44%)]	Loss: 1.766740
Train Epoch: 1 [26880/60000 (45%)]	Loss: 1.815301
Train Epoch: 1 [27520/60000 (46%)]	Loss: 1.718148
Train Epoch: 1 [28160/60000 (47%)]	Loss: 1.746439


KeyboardInterrupt: 